# Exploring GTSAM  Package for Pose Optimization with Factor Graphs

In [ ]:
%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

# import gtsampy as gtsam
import gtsam
import gtsam.utils.plot as gtsam_plot
import matplotlib.pyplot as plt
import numpy as np
from ipyvtklink.viewer import ViewInteractiveWidget
from vedo import  *
from utils import *

In [ ]:
import tensorflow as tf
from tensorflow import  sin, cos, tan

def R_tf(angs):
    """generates rotation matrix using euler angles
    angs = tf.constant(phi, theta, psi) (aka rot about (x,y,z))
            can be single set of angles or batch for multiple cells
    """

    if len(tf.shape(angs)) == 1:
        angs = angs[None,:]
    phi = angs[:,0]
    theta = angs[:,1]
    psi = angs[:,2]

    mat = tf.Variable([[cos(theta)*cos(psi), sin(psi)*cos(phi) + sin(phi)*sin(theta)*cos(psi), sin(phi)*sin(psi) - sin(theta)*cos(phi)*cos(psi)],
                       [-sin(psi)*cos(theta), cos(phi)*cos(psi) - sin(phi)*sin(theta)*sin(psi), sin(phi)*cos(psi) + sin(theta)*sin(psi)*cos(phi)],
                       [sin(theta), -sin(phi)*cos(theta), cos(phi)*cos(theta)]
                        ])

    mat = tf.transpose(mat, [2, 0, 1])
    mat = tf.squeeze(mat)
    return mat

# Establish Origin 

$ [x, y, z, \phi, \theta, \psi ] $

In [ ]:
p = np.array([0,0,0,0,0,0], dtype = np.float64)
start = gtsam.Pose3(p)
print(start)

# newpt = gtsam.Pose3(np.array([1,1,1,0,0,0], dtype = np.float64))

#  Debug Skip 1,3,5 dataset

In [ ]:
from matplotlib import pyplot as plt
plt, ax = plt.subplots(2,1)

startidx = 20
nframes = 1000
# odometry_history = np.load("test_data/leddartech_pixset/T_vec_history_1and3and5.npy")[:nframes,:6]
# ij = np.load("test_data/leddartech_pixset/T_vec_history_1and3and5.npy")[:nframes,6:]
# odometry_history = np.load("test_data/Ford_Campus_Dataset/T_vec_history_1and3and5.npy")[:nframes,:6]
# ij = np.load("test_data/Ford_Campus_Dataset/T_vec_history_1and3and5.npy")[:nframes,6:]
odometry_history = np.load("test_data/KITTI/117/T_vec_history_3and4and5.npy")[startidx:startidx+nframes,:6]
ij = np.load("test_data/KITTI/117/T_vec_history_3and4and5.npy")[startidx:startidx+nframes,6:]

# print(odometry_history)
# print(ij)
# ij = get_ij(tf.convert_to_tensor(ij))
# print(ij)

skips = ij[:,1] - ij[:,0]
# ones = np.where(skips == 1)
threes = np.where(skips == 3)
fours = np.where(skips == 4)
fives = np.where(skips == 5)

vf = odometry_history[:,0]/skips #forward velocity, normalized by num frames elapsed
# vf = odometry_history[:,-1]/skips #rotation, normalized by num frames elapsed
# vf[threes] = vf[threes]
# vf[fives] = vf[fives]*4/5
# ax[1].plot(ones[0], vf[ones])  #ones are the problem!!!??!!
ax[0].set_title("Forward Translation Estimate Per Thread")
ax[0].plot(fours[0], vf[fours], label = "every 4th frame")
ax[0].plot(fives[0], vf[fives], label = 'every 5th frame')
ax[0].plot(threes[0], vf[threes], label = 'every 3rd frame')
ax[0].legend(loc = 'best')

yaw = odometry_history[:,-1]/skips #rotation, normalized by num frames elapsed
ax[1].set_title("Yaw Estimate Per Thread")
ax[1].plot(fours[0], yaw[fours], label = "every 4th frame")
ax[1].plot(fives[0], yaw[fives], label = 'every 5th frame')
ax[1].plot(threes[0], yaw[threes], label = 'every 3rd frame')
ax[1].legend(loc = 'best')


# 3D Braided Odometry

In [ ]:
# real braided odometry via ICET ROS pkg ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

startidx = 20
nframes = 100
# odometry_history = np.load("test_data/leddartech_pixset/T_vec_history_1and3and5.npy")[:nframes,:]
# pred_stds_history = np.load("test_data/leddartech_pixset/cov_vec_history_1and3and5.npy")[:nframes,:]
# # odometry_history = np.load("test_data/Ford_Campus_Dataset/T_vec_history_1and3and5.npy")[startidx:nframes+startidx,:]
# # pred_stds_history = np.load("test_data/Ford_Campus_Dataset/cov_vec_history_1and3and5.npy")[startidx:nframes+startidx,:]
odometry_history = np.load("test_data/KITTI/117/T_vec_history_3and4and5.npy")[startidx:startidx+nframes]
#actual ICET covariance estimate
pred_stds_history =  np.load("test_data/KITTI/117/cov_vec_history_3and4and5.npy")[startidx:startidx+nframes]
#keep static
# pred_stds_history = np.tile(np.array([[0.001, 0.001, 0.001, 1e-6, 1e-6, 1e-6]]), (len(odometry_history),1))
# pred_stds_history = np.append(pred_stds_history, ij, axis= 1) #match format output by ROS package
ij = np.load("test_data/KITTI/117/T_vec_history_3and4and5.npy")[startidx:startidx+nframes,6:]
# print("ij before: \n", ij[:10])

#debug: only keep threes -------------------------
skips = ij[:,1] - ij[:,0]
threes = np.where(skips == 3)
# print(threes)
ij = ij[threes]
odometry_history = odometry_history[threes]
pred_stds_history = pred_stds_history[threes]
# ------------------------------------------------

ij = get_ij(ij.astype(np.int32)).numpy()
# print("ij after: \n", ij)

#raw Ford dataset ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# startidx = 800
# nframes = 500
# odometry_history = np.loadtxt("/home/derm/ASAR/v3/spherical_paper/FORD_results/ICET_estimates_ford.txt")[startidx:nframes+startidx,:]
# ij = np.append(np.arange(len(odometry_history))[:,None], np.arange(len(odometry_history))[:,None] + 1, axis = 1)
# pred_stds_history = np.tile(np.array([[0.001, 0.001, 0.001, 1e-6, 1e-6, 1e-6]]), (len(odometry_history),1))
# pred_stds_history = np.append(pred_stds_history, ij, axis= 1) #match format output by ROS package
# odometry_history = np.append(odometry_history, ij, axis= 1)   #match format output by ROS package

#simplified data ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# odometry_history = np.array([[1., 1, 0, 0, 0, 0., 0, 1],
#                              [1., 1, 0, 0, 0, 0., 1, 2],
#                              [1., 1, 0, 0, 0, 0., 2, 3], 
#                              [0., 1, 0, 0, 0, 0.0, 1, 4], #add conflicting measurement
#                              [1., 1, 0, 0, 0, 0., 3, 4] ]) 
# ij = odometry_history[:,-2:].astype(int)
# print(ij)

# npts = 200
# odometry_history = np.tile(np.array([0.0, 0.05, 0.002, 0.0, 0.0, 0.1]), (npts,1))
# #add process noise
# # odometry_history += np.random.randn(np.shape(odometry_history)[0], 
# #                                     np.shape(odometry_history)[1])*np.array([0,0,0, 0.0,0.1,0.])  
# pred_stds_history = np.tile(np.array([[0.001, 0.005, 0.001, 1e-6, 1e-6, 1e-5]]), (len(odometry_history),1))
# # pred_stds_history = np.tile(np.array([[0.001, 0.001, 0.001, 1e-6, 1e-6, 1e-6]]), (len(odometry_history),1))
# ij = np.array([[0,1]])
# for i in range(1,len(odometry_history)):
#     ij = np.append(ij, np.array([[i, i+1]]), axis = 0)
# #mess with last constraint (add loop closure)
# # ij[-1,1] = 50
# # ij = np.append(np.array([[0,15]]), ij[:-1], axis = 0)
# odometry_history = np.append(odometry_history, ij, axis= 1)   #match format output by ROS package
# pred_stds_history = np.append(pred_stds_history, ij, axis= 1) #match format output by ROS package

# braided corkscrew ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# num_nodes = 50 #total number of poses in graph
# skips = 5 #number of frames to skip for low frequency registration
# odometry_history = np.tile(np.array([0.0, 0.05, 0.002, 0.0, 0.0, 0.1]), (num_nodes,1))
# pred_stds_history = np.tile(np.array([[0.001, 0.001, 0.001, 1e-6, 1e-6, 1e-6]]), (len(odometry_history),1))
# ij = np.array([[0,1]])
# #get first constraints
# for i in range(1,len(odometry_history)):
#     ij = np.append(ij, np.array([[i, i+1]]), axis = 0)
# for j in np.arange(1,len(odometry_history) - skips, skips):
#     ij = np.append(ij, np.array([[j, j+skips]]), axis = 0)
#     skip_odom =  skips*np.array([[0.0, 0.05, 0.002, 0.0, 0.0, 0.1]]) #simple linearization
# #     skip_odom =  np.array([[0.1, 0.24, 0.006, 0.0, 0.0, 0.6]]) 
#     odometry_history = np.append(odometry_history, skip_odom, axis = 0)
#     skip_pred_stds = np.array([[0.001, 0.001, 0.001, 1e-6, 1e-6, 1e-6]])
#     pred_stds_history = np.append(pred_stds_history, skip_pred_stds, axis = 0)   
# odometry_history = np.append(odometry_history, ij, axis= 1)   #match format output by ROS package
# pred_stds_history = np.append(pred_stds_history, ij, axis= 1) #match format output by ROS package

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# print(ij)
# print(odometry_history)

# Single Batch Optimization

#### Notes:

Falls apart with > 50 frames at near-zero initial conditions

"Indeterminate system" error if > 200 frames, even from a single thread 


In [ ]:
from utils import get_ij

# Create an empty nonlinear factor graph
graph = gtsam.NonlinearFactorGraph() 

# Add a prior on the first pose, setting it to the origin
# A prior factor consists of a mean and a noise model (covariance matrix)
# priorMean = gtsam.Pose3(np.zeros([6,1], dtype = np.float64))  # prior at origin
priorRot = gtsam.Rot3(R_tf(np.array([0., 0., 0.8])))
priorMean = gtsam.Pose3(priorRot, np.array([0., 0., .0])) #prior at nonzero pose
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(
    np.array([0.003, 0.003, 0.003, 1e-5, 1e-5, 1e-5], dtype = np.float64))
# firstKey = initial.keys()[0] #TODO: what does this do???
graph.add(gtsam.PriorFactorPose3(0, priorMean, PRIOR_NOISE)) #constrain first point at priorMean

#loop through all constraints 
for i in range(len(ij)):
# for i in range(int(np.max(ij))):
# for i in range(1,int(np.max(ij)) + 1): #test
    #get constraint 
    raw_odom = odometry_history[i,:-2] #omit scan indices stored at end of line
    #convert to Point3 strucutre
    point = raw_odom[:3] 
    rot = gtsam.Rot3(R_tf(raw_odom[3:]))
    odometry_estimate = gtsam.Pose3(rot, point)
#     print("\n odometry estimate:\n", odometry_estimate)
    cov_estimate = gtsam.noiseModel.Diagonal.Sigmas(pred_stds_history[i,:-2])
#     cov_estimate = gtsam.noiseModel.Diagonal.Sigmas(pred_stds_history[i,:]) #dont need to omit scan indices for fake data
    first_idx = ij[i,0]
    second_idx = ij[i,1] 
#     print(first_idx, second_idx) #misses last element in ij!!!
    graph.add(gtsam.BetweenFactorPose3(first_idx, second_idx, odometry_estimate, cov_estimate))

# print(graph.size())
#set to zero initial conditions
initial = gtsam.Values()
# print(initial.keys())
# print("\n graph size:", graph.size())
# print(int(np.max(ij)))
# for j in range(graph.size()):
for j in range(int(np.max(ij))+1):
#     zero_rot = gtsam.Rot3(R_tf(np.array([0., 0., 0.])))
#     zero_pose3 = gtsam.Pose3(zero_rot, np.array([0., 0., 0.])) #does not work with zero initial conds??
#     initial.insert(j, zero_pose3)
    init_rot = gtsam.Rot3(R_tf(0.01*np.random.randn(3)))
    init_pose = gtsam.Pose3(init_rot, np.random.randn(3))
    initial.insert(j, init_pose)
    
# # optimize using Levenberg-Marquardt optimization
# # damped optimizer - seems to work much better here
# params = gtsam.LevenbergMarquardtParams()
# optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)

# # simple gauss newton - kinda unreliable for high DOF problems (especially with zero initial conditions)
# params = gtsam.GaussNewtonParams()
# params.setVerbosity("Termination")  # this will show info about stopping conds
# optimizer = gtsam.GaussNewtonOptimizer(graph, initial, params)

# dogleg
params = gtsam.DoglegParams()
params.setVerbosity("Termination")
params.setMaxIterations(100)
optimizer = gtsam.DoglegOptimizer(graph, initial, params)

result = optimizer.optimize()
marginals = gtsam.Marginals(graph, result) #calculate covariance estimates for each pose

print("initial error = ", graph.error(initial))
print("final error = ", graph.error(result))
# print(initial.keys())
# print(graph.keys())

In [ ]:
#Plot results
plt1 = Plotter(N = 1, axes = 1, bg =(1, 1, 1), interactive = True) #ax=7 gives rulers
disp = []
# disp = plot_results(disp, result, ij, draw_axis=True)
disp = plot_results(disp, result, ij, marginals, draw_axis = True) #draws covarince ellipsoids
plt1.show(disp, "Factor Graph Test")
ViewInteractiveWidget(plt1.window) #need to comment out when working on laptop

In [ ]:
def draw_ell(disp, frame, sigma, pc = 1, alpha = 0.5):
    """draw distribution ellipses given mu and sigma tensors"""
    color = [0.3, 0.3, 0.8]

    mu = frame[:3,-1]
    eig = np.linalg.eig(sigma[:,:].numpy())
    eigenval = eig[0] #correspond to lengths of axis
    eigenvec = eig[1]

    #eigenvals not ordered in decreasing size when scale large enough
    a1 = tf.sort(eigenval).numpy()[-1]
    a2 = tf.sort(eigenval).numpy()[-2]
    a3 = tf.sort(eigenval).numpy()[-3]
#     a1 = eigenval[0]
#     a2 = eigenval[1]
#     a3 = eigenval[2]
    
    #only need 3x3 of eigenvector to describe rotation of covariance ellipse
    eigenvec = tf.gather(eigenvec, tf.argsort(eigenval, direction='DESCENDING'), axis = 1)
#     eigenvec = tf.gather(eigenvec, tf.argsort(eigenval, direction='DESCENDING'), axis = 0)

    #rotate into frame of previous odometry estimate (instead of world xyz frame)     
#         eigenvec = eigenvec @ frame[0,:3,:3]  #frame has axis switched up in some frames
#         eigenvec = eigenvec @ tf.gather(frame[0,:3,:3], tf.argsort(eigenval, direction='DESCENDING'), axis = 1)      
    #DEBUG- issues because each frame is not centered w.r.t origin??
    rot = frame[:3,:3]
#         rot = -tf.linalg.pinv(frame[:3,:3])
#     print("\n rot \n:",  rot)
#     print("\n eigenvec before: \n", eigenvec)
    eigenvec = eigenvec @ rot #works(ish) if I comment this out ...
#         print("\n eigenvec after: \n", eigenvec)

#     #DEBUG--------------------------
#     scale = 2
#     rot = frame[:3,:3]
# #         E = np.eye(3) #TODO: need to switch order of rows with a1,a2,a3
#     correct_order = tf.argsort(np.linalg.eig(sigma[:,:].numpy())[0], direction = "DESCENDING").numpy()
# #         correct_order = tf.argsort(np.linalg.eig(sigma[:,:].numpy())[0]).numpy()
# #     E = np.eye(3)[correct_order]
#     E = np.eye(3)
#     A = rot @ E * scale * np.sqrt(np.array([a1, a2, a3]))
#     xvec = Arrow(frame[:3,-1], frame[:3,-1] + A[:,0], c = 'red', alpha = alpha) 
#     yvec = Arrow(frame[:3,-1], frame[:3,-1] + A[:,1], c = 'green', alpha = alpha) 
#     zvec = Arrow(frame[:3,-1], frame[:3,-1] + A[:,2], c = 'blue', alpha = alpha) 
#     disp.append(xvec)
#     disp.append(yvec)
#     disp.append(zvec)
#     # -------------------------------

    if mu[0] != 0 and mu[1] != 0:
        ell = Ell(pos=(mu[0], mu[1], mu[2]), axis1 = 4*np.sqrt(abs(a1)), 
        axis2 = 4*np.sqrt(abs(a2)), axis3 = 4*np.sqrt(abs(a3)), 
#         angs = (np.array([R2Euler(rot)[0], R2Euler(rot)[1], R2Euler(rot)[2] ])), c=color, alpha=alpha, res=12)
        angs = (np.array([R2Euler(eigenvec)[0], R2Euler(eigenvec)[1], R2Euler(eigenvec)[2] ])), c=color, alpha=alpha, res=12)
        disp.append(ell)

    return(disp)

In [ ]:
def plot_results(disp, results, ij, marginals = None, draw_axis = False):
    
    #plot coordinates of point centers
    for i in range(result.size()): #loop through all elements in results
        p = result.atPose3(i).matrix()
        p_center = Points([[p[0,-1], p[1,-1],  p[2,-1]]], r = 5)
        disp.append(p_center)
        #add text labels
        t = Text3D(i, p[:3,-1], s = 0.01, alpha = 0.3).followCamera() #need for deskop
#         t = Text3D(i, p[:3,-1], s = 0.01, alpha = 0.3).follow_camera() #newer Vedo on Laptop?
        disp.append(t)
        if draw_axis is True:
            disp = draw_body_frame_axis(p, disp) #draw coordinate frames
        if marginals and i < len(ij): #num marginals in a chain = length of chain -1
            frame = result.atPose3(ij[i,0]).matrix()#[None,:]
            sigma = marginals.marginalCovariance(i)[:3,:3] #[None,:3,:3] #just ignore parts of sigma related to rotations
            sigma = tf.convert_to_tensor(sigma)
            disp = draw_ell(disp, frame, sigma)
            #TODO: plot last ellispe
            
    #draw constraints using ij
    for c in range(len(ij)):
        pt1 = results.atPose3(ij[c,0]).translation()   #get coords of results i and j 
        pt2 = results.atPose3(ij[c,1]).translation()
        L = Line(p0 = pt1, p1 = pt2, lw = 1)
        disp.append(L)
        
    #draw covarince ellipsoids for each pose
    #TODO: transform to respective frame (CURRENTLY PLOTTING IN GLOBAL XYZ!!!)
#     if marginals:
#         for k in range(0, int(np.max(ij))):
#             frame = result.atPose3(ij[k,0]).matrix()[None,:] #coordinate frame of pose i
# #             frame = result.atPose3(ij[k,1]).matrix()[None,:] #draw ellispse at subsequent point j
# #             sigma = marginals.marginalCovariance(k)[None,:,:] #was this
#             sigma = marginals.marginalCovariance(k)[None,:3,:3] #just ignore parts of sigma related to rotations
#             sigma = tf.convert_to_tensor(sigma)
#             disp = draw_ell(disp, frame, sigma)
#             print("--------- frame ", k, "---------------")
#             print(frame)
#             print("\n sigma: \n", np.sqrt(np.diag(marginals.marginalCovariance(k))))
                                                 
    return(disp)
    

In [ ]:
def draw_body_frame_axis(x, disp = []):
    """draws local xyz axis arrows on each pose in x """
    scale = 0.25 #axis length
    alpha = 1
#     print(x)
    rot = x[:3,:3]
    A = rot @ np.eye(3) * scale
    xvec = Arrow(x[:3,-1], x[:3,-1] + A[:,0], c = 'red', alpha = alpha) 
    yvec = Arrow(x[:3,-1], x[:3,-1] + A[:,1], c = 'green', alpha = alpha) 
    zvec = Arrow(x[:3,-1], x[:3,-1] + A[:,2], c = 'blue', alpha = alpha) 
    disp.append(xvec)
    disp.append(yvec)
    disp.append(zvec)
    return(disp)

In [ ]:
class Ell(Mesh):
    """
    Build a 3D ellipsoid centered at position `pos`.

    |projectsphere|

    |pca| |pca.py|_
    """
    def __init__(self, pos=(0, 0, 0), axis1= 1, axis2 = 2, axis3 = 3, angs = np.array([0,0,0]),
                 c="cyan4", alpha=1, res=24):

        self.center = pos
        self.va_error = 0
        self.vb_error = 0
        self.vc_error = 0
        self.axis1 = axis1
        self.axis2 = axis2
        self.axis3 = axis3
        self.nr_of_points = 1 # used by pcaEllipsoid

        if utils.isSequence(res): #new Vedo
#         if utils.is_sequence(res): #old Vedo
            res_t, res_phi = res
        else:
            res_t, res_phi = 2*res, res

        elliSource = vtk.vtkSphereSource()
        elliSource.SetThetaResolution(res_t)
        elliSource.SetPhiResolution(res_phi)
        elliSource.Update()
        l1 = axis1
        l2 = axis2
        l3 = axis3
        self.va = l1
        self.vb = l2
        self.vc = l3
        axis1 = 1
        axis2 = 1
        axis3 = 1
        angle = angs[0] #np.arcsin(np.dot(axis1, axis2))
        theta = angs[1] #np.arccos(axis3[2])
        phi =  angs[2] #np.arctan2(axis3[1], axis3[0])

        t = vtk.vtkTransform()
        t.PostMultiply()
        t.Scale(l1, l2, l3)

        #needed theta and angle to be negative before messing with E_xz, E_yz...
        t.RotateZ(np.rad2deg(phi))
        t.RotateY(-np.rad2deg(theta)) #flipped sign here 5/19
        t.RotateX(-np.rad2deg(angle)) #flipped sign here 5/19
        
        tf = vtk.vtkTransformPolyDataFilter()
        tf.SetInputData(elliSource.GetOutput())
        tf.SetTransform(t)
        tf.Update()
        pd = tf.GetOutput()
        self.transformation = t

        Mesh.__init__(self, pd, c, alpha)
        self.phong()
        self.GetProperty().BackfaceCullingOn()
        self.SetPosition(pos)
        self.Length = -np.array(axis1) / 2 + pos
        self.top = np.array(axis1) / 2 + pos
        self.name = "Ell"

In [ ]:
def R2Euler(mat):
    """determines euler angles from euler rotation matrix"""

    if len( tf.shape(mat) ) == 2:
        mat = mat[None, :, :]

    R_sum = np.sqrt(( mat[:,0,0]**2 + mat[:,0,1]**2 + mat[:,1,2]**2 + mat[:,2,2]**2 ) / 2)

    phi = np.arctan2(-mat[:,1,2],mat[:,2,2])
    theta = np.arctan2(mat[:,0,2], R_sum)
    psi = np.arctan2(-mat[:,0,1], mat[:,0,0])

    angs = np.array([phi, theta, psi])
    return angs

In [ ]:
#DEBUG - [x, y, z, phi, theta, psi] -> Pose3
# t = np.array([1, 2, 3, 0.0 ,0.0 ,0.1])
# point = t[:3] 
# rot = gtsam.Rot3(R_tf(t[3:]))
# Pose = gtsam.Pose3(rot, point)
# print(Pose)

print(marginals)

# 2D Odometry Example

In [ ]:
# Create noise models
ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
ODOMETRY_NOISE2 = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.5])) #test
# PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.05, 0.05, 0.05])) #test


# Create an empty nonlinear factor graph
graph = gtsam.NonlinearFactorGraph()

# Add a prior on the first pose, setting it to the origin
# A prior factor consists of a mean and a noise model (covariance matrix)
priorMean = gtsam.Pose2(0.0, 0.0, 0.0)  # prior at origin
graph.add(gtsam.PriorFactorPose2(1, priorMean, PRIOR_NOISE))

# Add odometry factors
odometry = gtsam.Pose2(2.0, 0.0, 0.2)
# For simplicity, we will use the same noise model for each odometry factor
# Create odometry (Between) factors between consecutive poses
graph.add(gtsam.BetweenFactorPose2(1, 2, odometry, ODOMETRY_NOISE))
graph.add(gtsam.BetweenFactorPose2(2, 3, odometry, ODOMETRY_NOISE2))
graph.add(gtsam.BetweenFactorPose2(3, 4, odometry, ODOMETRY_NOISE2)) #test
# print("\nFactor Graph:\n{}".format(graph))

# Create the data structure to hold the initialEstimate estimate to the solution
# For illustrative purposes, these have been deliberately set to incorrect values
initial = gtsam.Values()
initial.insert(1, gtsam.Pose2(0.5, 0.0, 0.2))
initial.insert(2, gtsam.Pose2(2.3, 0.1, -0.2))
initial.insert(3, gtsam.Pose2(4.1, 0.1, 0.1))
initial.insert(4, gtsam.Pose2(4.1, 0.1, 0.1))#test
# print("\nInitial Estimate:\n{}".format(initial))

#used for 2D
# optimize using Levenberg-Marquardt optimization
params = gtsam.LevenbergMarquardtParams() 
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)

result = optimizer.optimize()
# print("\nFinal Result:\n{}".format(result))

# 5. Calculate and print marginal covariances for all variables
marginals = gtsam.Marginals(graph, result)
# for i in range(1, 4):
#     print("X{} covariance:\n{}\n".format(i,
#                                          marginals.marginalCovariance(i)))

for i in range(1, 5):
    gtsam_plot.plot_pose2(0, result.atPose2(i), 0.5,
                          marginals.marginalCovariance(i))
plt.axis('equal')
plt.show()

In [ ]:
#test plotting 2D odometry estimates
# a = gtsam.Pose2(0.5, 0.0, 0.2)

print(result.atPose2(2))


# Pose3 SLAM Example

g2o is a file format for representing human-readable graphs for optimization problems
https://github.com/uoip/g2opy

In [ ]:
g2oFile = "gtsampy/Data/pose3example.txt"
# g2oFile = gtsam.findExampleDataFile("pose3example.txt")
is3D = True
graph, initial  = gtsam.readG2o(g2oFile, is3D)
# print(initial)

priorModel = gtsam.noiseModel.Diagonal.Variances(
        np.array([1e-6, 1e-6, 1e-6, 1e-4, 1e-4, 1e-4]))

firstKey = initial.keys()[0]
graph.add(gtsam.PriorFactorPose3(0, gtsam.Pose3(), priorModel))

initialization = gtsam.InitializePose3.initialize(graph)
# print(initialization)